### Lorentz System

$\frac{\mathrm{d}x}{\mathrm{d}t} = \sigma (y - x), \\[6pt]$


$\frac{\mathrm{d}y}{\mathrm{d}t} = x (\rho - z) - y, \\[6pt]$


$\frac{\mathrm{d}z}{\mathrm{d}t} = x y - \beta z.$

$σ = 10$, $β = 8/3$ and $ρ = 28$

In [1]:
import autograd.numpy as np
from autograd import grad, elementwise_grad
import autograd.numpy.random as npr
from matplotlib import pyplot as plt

### Activation Functions

In [2]:
def sigmoid(z) -> float:
    return 1 / (1 + np.exp(-z))


def tanh(z) -> float:
    return (np.exp(z) - np.exp(-z)) / (np.exp(z) + np.exp(-z))


def elu(z, alpha: float) -> float:
    return alpha * (np.exp(z) - 1) if z < 0 else z

### Trial Solution & Analytical Solution

In [8]:
def trial_solution(
    input: np.array, parameters: np.array, initial_conditions: np.array
) -> np.array:
    return initial_conditions + input * neural_network(parameters, input)


def right_hand_side(
    input: np.array, trial_sol: np.array, constants: np.array
) -> np.array:
    return np.array(
        [
            constants[0] * (trial_sol[1] - trial_sol[0]),
            trial_sol[0] * (constants[2] - trial_sol[2]) - trial_sol[1],
            trial_sol[0] * trial_sol[1] - constants[1] * trial_sol[2],
        ]
    )


def lorenz(xyz, *, s=10, r=28, b=2.667):
    """
    Parameters
    ----------
    xyz : array-like, shape (3,)
       Point of interest in three-dimensional space.
    s, r, b : float
       Parameters defining the Lorenz attractor.

    Returns
    -------
    xyz_dot : array, shape (3,)
       Values of the Lorenz attractor's partial derivatives at *xyz*.
    """
    x, y, z = xyz
    x_dot = s * (y - x)
    y_dot = r * x - y - x * z
    z_dot = x * y - b * z
    return np.array([x_dot, y_dot, z_dot])

### Cost Function

In [9]:
def MSE_loss_function(input, parameters, constants: np.array):
    trial_sol = trial_solution(input, parameters)
    prediction = right_hand_side(input, trial_sol, constants)
    neural_network_gradient = elementwise_grad(neural_network, 0)(input, parameters)

    error_squared = (neural_network_gradient - prediction) ** 2
    loss_sum = np.sum(error_squared)

    return loss_sum / np.shape(error_squared)[0]

### Neural Network

In [ ]:
def neural_network(input, parameters):
    